# Banco de dados integrado

In [33]:
import sqlite3
banco = sqlite3.connect('banco.db')
cursor = banco.cursor()

cursor.execute('CREATE TABLE IF NOT EXISTS Users (name VARCHAR(255), email VARCHAR(255) UNIQUE, ativo BOOLEAN);')
banco.commit()

In [5]:
cursor.execute('CREATE TABLE IF NOT EXISTS Products (id INTEGER PRIMARY KEY AUTOINCREMENT, title VARCHAR(255), price REAL, ativo BOOLEAN);')
banco.commit()

In [8]:
cursor.execute('INSERT INTO Users (name, email, ativo) VALUES ("Ciro", "cironass@mail.com", TRUE);')
cursor.execute('INSERT INTO Users (name, email, ativo) VALUES ("Outro", "outro@mail.com", FALSE);')
banco.commit()

In [26]:
nome = "Ciro"
email = "cirogn@mail.com"
status = 1
cursor.execute(f'INSERT INTO Users (name, email, ativo) VALUES ("{nome}", "{email}", "{status}");')
banco.commit()

In [29]:
cursor.execute('UPDATE Users set email = "ciroguilhermenass@mail.com" where name = "Ciro";')
banco.commit()

In [31]:
cursor.execute('DELETE FROM Users where ativo = "0";')
banco.commit()

In [34]:
cursor.execute('SELECT * FROM Users;')
resultado = cursor.fetchall()
print(resultado) # imprime tupla
for user in resultado: # percorre a tupla e imprime valores das posições respectivas
    print(f"Usuario: {user[0]}, Email: {user[1]}, Ativo: {user[2]}")

[('Ciro', 'ciroguilhermenass@mail.com', 1), ('Ciro', 'ciroguilhermenass@mail.com', 1), ('Ciro', 'ciroguilhermenass@mail.com', 1), ('Ciro', 'ciroguilhermenass@mail.com', 'True'), ('Ciro', 'ciroguilhermenass@mail.com', 1), ('Ciro', 'ciroguilhermenass@mail.com', 1)]
Usuario: Ciro, Email: ciroguilhermenass@mail.com, Ativo: 1
Usuario: Ciro, Email: ciroguilhermenass@mail.com, Ativo: 1
Usuario: Ciro, Email: ciroguilhermenass@mail.com, Ativo: 1
Usuario: Ciro, Email: ciroguilhermenass@mail.com, Ativo: True
Usuario: Ciro, Email: ciroguilhermenass@mail.com, Ativo: 1
Usuario: Ciro, Email: ciroguilhermenass@mail.com, Ativo: 1


In [1]:
import sqlite3
bancoagenda = sqlite3.connect('agenda.db')
cursoragenda = bancoagenda.cursor()
        
# cria a tabela de contatos se não existir
cursoragenda.execute('''
CREATE TABLE IF NOT EXISTS contacts (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    phone TEXT,
    email TEXT UNIQUE
);
''')
bancoagenda.commit()

# Funções da agenda

def add_contact(name, phone=None, email=None):
    """Adiciona um contato. Retorna o id inserido ou None em caso de erro."""
    try:
        cursoragenda.execute('INSERT INTO contacts (name, phone, email) VALUES (?, ?, ?)', (name, phone, email))
        bancoagenda.commit()
        return cursoragenda.lastrowid
    except sqlite3.IntegrityError as e:
        print('Erro ao adicionar contato:', e)
        return None

def update_contact(contact_id, name=None, phone=None, email=None):
    """Atualiza os campos fornecidos do contato com id = contact_id. Retorna True se atualizou."""
    fields = []
    params = []
    if name is not None:
        fields.append('name = ?'); params.append(name)
    if phone is not None:
        fields.append('phone = ?'); params.append(phone)
    if email is not None:
        fields.append('email = ?'); params.append(email)
    if not fields:
        return False
    params.append(contact_id)
    sql = 'UPDATE contacts SET ' + ', '.join(fields) + ' WHERE id = ?'
    cursoragenda.execute(sql, params)
    bancoagenda.commit()
    return cursoragenda.rowcount > 0

def delete_contact(contact_id):
    """Deleta um contato pelo id. Retorna True se deletou."""
    cursoragenda.execute('DELETE FROM contacts WHERE id = ?', (contact_id,))
    bancoagenda.commit()
    return cursoragenda.rowcount > 0

def list_contacts():
    """Retorna a lista de contatos (id, name, phone, email)."""
    cursoragenda.execute('SELECT id, name, phone, email FROM contacts ORDER BY name')
    return cursoragenda.fetchall()

# Demonstração rápida (rodar a célula para ver o comportamento)
print('--- Demo da agenda de contatos ---')
# Adiciona exemplos (pode falhar se o email já existir)
id1 = add_contact('Alice', '1111-2222', 'alice@mail.com')
id2 = add_contact('Bob', '3333-4444', 'bob@mail.com')
print('IDs adicionados:', id1, id2)

print('\nLista de contatos:')
for c in list_contacts():
    print(c)

# Atualiza telefone do primeiro contato (se existir)
if id1:
    update_contact(id1, phone='9999-0000')
    print('\nApós atualização:')
    for c in list_contacts():
        print(c)

# Deleta segundo contato (se existir)
if id2:
    delete_contact(id2)
    print('\nApós deleção:')
    for c in list_contacts():
        print(c)

# Obs: manter a conexão aberta no notebook facilita operações interativas
# Para fechar explicitamente use: bancoagenda.close()


--- Demo da agenda de contatos ---
Erro ao adicionar contato: UNIQUE constraint failed: contacts.email
IDs adicionados: None 3

Lista de contatos:
(1, 'Alice', '9999-0000', 'alice@mail.com')
(3, 'Bob', '3333-4444', 'bob@mail.com')

Após deleção:
(1, 'Alice', '9999-0000', 'alice@mail.com')
